In [1]:
#import libraries
import geopandas as gpd
import ee
import geemap
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
# Intialize Earth Engine
ee.Authenticate()
ee.Initialize(project="ee-fajarramadhana505")

In [15]:
# -----------------------------------------------------------------------------
# CELL 1: Area of Interest and Time Range Setup
# -----------------------------------------------------------------------------
# The JavaScript 'Brunei' object is not a standard Earth Engine geometry.
# We'll create a placeholder for a sample region. You can replace this with
# the actual geometry you intend to use. Here, we are creating a rectangle
# around the approximate location of Brunei.
brunei_coords = [114.0, 4.0, 115.5, 5.5]
Brunei = ee.Geometry.Rectangle(brunei_coords)

# Develop the Brunei point and buffer
geom = ee.Geometry.Point(102.14, -0.26).buffer(10000)

# Define the start and end years for the 10-year period
start_year_range = 2015  # The first year of the dNBR calculation
end_year_range = 2024    # The last year for the second image in the pair

In [16]:
# -----------------------------------------------------------------------------
# CELL 2: Data Collection and Cloud Masking Function
# -----------------------------------------------------------------------------
# Input Landsat collections
l9sr = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
l8sr = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
l5sr = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
lsr = ee.ImageCollection(l9sr.merge(l8sr).merge(l5sr))

# Function to cloud mask from the Fmask band of Landsat SR data
def maskLsr(img):
    qa = img.select('QA_PIXEL')
    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4
    
    mask = qa.bitwiseAnd(dilated).eq(0) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
        
    return img.updateMask(mask)

In [22]:
# -----------------------------------------------------------------------------
# CELL 3: Automated dNBR Calculation and Export Loop
# -----------------------------------------------------------------------------
# Loop through the years to calculate dNBR for each pair of consecutive years
for year in range(start_year_range, end_year_range):
    print(f"Processing dNBR for years {year} and {year + 1}...")

    # Calculate composite for the first year (e.g., 2015)
    startDate1 = ee.Date.fromYMD(year, 1, 1)
    endDate1 = ee.Date.fromYMD(year + 1, 1, 1)
    composite1 = lsr.filterDate(startDate1, endDate1) \
                    .map(maskLsr) \
                    .median() \
                    .multiply(0.0000275) \
                    .add(-0.2)
    
    # Calculate composite for the second year (e.g., 2016)
    startDate2 = ee.Date.fromYMD(year + 1, 1, 1)
    endDate2 = ee.Date.fromYMD(year + 2, 1, 1)
    composite2 = lsr.filterDate(startDate2, endDate2) \
                    .map(maskLsr) \
                    .median() \
                    .multiply(0.0000275) \
                    .add(-0.2)
    
    # Calculate pre-fire and post-fire NBR for the year pair
    BurntIndices1 = composite1.expression(
        "(NIR - SWIR2) / (NIR + SWIR2)",
        {
            'SWIR2': composite1.select("SR_B7"),
            'NIR': composite1.select("SR_B5"),
        })

    BurntIndices2 = composite2.expression(
        "(NIR - SWIR2) / (NIR + SWIR2)",
        {
            'SWIR2': composite2.select("SR_B7"),
            'NIR': composite2.select("SR_B5"),
        })

    # Calculate dNBR
    dNBR_unscaled = BurntIndices1.subtract(BurntIndices2)
    dNBR_scaled = dNBR_unscaled.multiply(1000).clip(Brunei)

Processing dNBR for years 2015 and 2016...
Processing dNBR for years 2016 and 2017...
Processing dNBR for years 2017 and 2018...
Processing dNBR for years 2018 and 2019...
Processing dNBR for years 2019 and 2020...
Processing dNBR for years 2020 and 2021...
Processing dNBR for years 2021 and 2022...
Processing dNBR for years 2022 and 2023...
Processing dNBR for years 2023 and 2024...


In [25]:
# Export images to Drive
for year in range(start_year_range, end_year_range):    
    description_unscaled = f'dNBR_unscaledBrunei_{year}_{year+1}'
    task_unscaled = ee.batch.Export.image.toDrive(
        image=dNBR_unscaled,
        description=description_unscaled,
        folder='dNBR_unscaled',
        maxPixels=1e13,
        region=Brunei.getInfo()['coordinates'],
        scale=30
    )
    task_unscaled.start()
    print(f"Export task for {description_unscaled} started: {task_unscaled.id}")

    description_scaled = f'dNBR_scaled_{year}_{year+1}'
    task_scaled = ee.batch.Export.image.toDrive(
        image=dNBR_scaled,
        description=description_scaled,
        folder='dNBR_scaled',
        maxPixels=1e13,
        region=Brunei.getInfo()['coordinates'],
        scale=30
    )
    task_scaled.start()
    print(f"Export task for {description_scaled} started: {task_scaled.id}")

print("All dNBR tasks have been submitted.")

Export task for dNBR_unscaledBrunei_2015_2016 started: TEP3XANNDD7URVSOQ34G266F
Export task for dNBR_scaled_2015_2016 started: VPD26BQZCNMDCRLTQW337VWJ
Export task for dNBR_unscaledBrunei_2016_2017 started: TX37DHQV6CU3T5KH727XJOKG
Export task for dNBR_scaled_2016_2017 started: IKEIJLNIEGX3RRPR7J22WZR3
Export task for dNBR_unscaledBrunei_2017_2018 started: QDBMPWF6GRMBAQJSRNFH5RDA
Export task for dNBR_scaled_2017_2018 started: SMWA2D3EPSUYJFRC6OIVQIJ2
Export task for dNBR_unscaledBrunei_2018_2019 started: OFJWLBQNG2ZHDIBT3TINRWL4
Export task for dNBR_scaled_2018_2019 started: LVWL3SW5J6HOVHNUHUD22PDK
Export task for dNBR_unscaledBrunei_2019_2020 started: FT6XRNQXT3KNGIGYWHPDWM3A
Export task for dNBR_scaled_2019_2020 started: RUI2AEURZDPAI6KBBCAVN26Z
Export task for dNBR_unscaledBrunei_2020_2021 started: RDLPP2OI7C7WIJ5N276MGE35
Export task for dNBR_scaled_2020_2021 started: SFIYVVDC7JDO3JRUOPXOH57W
Export task for dNBR_unscaledBrunei_2021_2022 started: AM34XQ7JQFRU2YINTTBTCPV3
Export t